In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import os
sys.path.insert(0, os.path.abspath('..'))
from secret import AK_SK
keys = AK_SK('../qianfan.keys')
os.environ["QIANFAN_ACCESS_KEY"] = keys.get_ak()
os.environ["QIANFAN_SECRET_KEY"] = keys.get_sk()

In [4]:
from myqianfan import QianfanLLM, QianfanEmbedding
from myqianfan import model_spec

llm = QianfanLLM(model_spec=model_spec.ERNIE4_Turbo8K, temperature=0.5)

[WARNING][2024-10-15 21:44:02.194] redis_rate_limiter.py:21 [t:8703163072]: no redis installed, RedisRateLimiter unavailable


In [25]:
from atlassian import Confluence
from langchain import hub
from langchain_chroma import Chroma
from langchain_core.output_parsers.transform import BaseTransformOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import ConfluenceLoader
from langchain.schema import BaseOutputParser


In [6]:
vectorstore = Chroma("qianfan", embedding_function=QianfanEmbedding(), persist_directory="vectorstore/fintopia.vec")

In [ ]:
# vectorstore.as_retriever()

## 第一次初始化时运行

In [8]:
def get_children_pages_recursively(client, page_id: str):
    child_pages = client.get_page_child_by_type(page_id)
    for page in child_pages:
        yield page
        if "id" in page:
            yield from get_children_pages_recursively(client, page["id"])

from getpass import getpass
wiki_psw = getpass("wiki:")
# get all pages under some page
confluence = Confluence(
    url='https://wiki.fintopia.tech/',
    username='tongxinwen@fintopia.tech',
    password=wiki_psw)
child_pages = [p for p in get_children_pages_recursively(confluence, "74290517")]

loader = ConfluenceLoader(
    url="https://wiki.fintopia.tech/",
    page_ids=[p["id"] for p in child_pages],
    username="tongxinwen@fintopia.tech",
    api_key=wiki_psw,
    limit=10,
)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=100)
splits = text_splitter.split_documents(documents)

batch_size = 16
doc_ids = []
for i in range(0, len(splits), batch_size):
    print(f"Adding documents {i} to {i+batch_size}")
    texts = [doc.page_content for doc in splits[i:i+batch_size]]
    metadatas = [doc.metadata for doc in splits[i:i+batch_size]]
    doc_ids += vectorstore.add_texts(texts, metadatas)

Adding documents 0 to 16
Adding documents 16 to 32
Adding documents 32 to 48
Adding documents 48 to 64
Adding documents 64 to 80
Adding documents 80 to 96
Adding documents 96 to 112
Adding documents 112 to 128
Adding documents 128 to 144
Adding documents 144 to 160
Adding documents 160 to 176
Adding documents 176 to 192
Adding documents 192 to 208
Adding documents 208 to 224
Adding documents 224 to 240
Adding documents 240 to 256
Adding documents 256 to 272
Adding documents 272 to 288
Adding documents 288 to 304
Adding documents 304 to 320
Adding documents 320 to 336
Adding documents 336 to 352
Adding documents 352 to 368
Adding documents 368 to 384
Adding documents 384 to 400
Adding documents 400 to 416
Adding documents 416 to 432
Adding documents 432 to 448
Adding documents 448 to 464
Adding documents 464 to 480
Adding documents 480 to 496
Adding documents 496 to 512


In [21]:
# vectorstore.search("数据库", search_type="similarity")

[Document(metadata={'id': '95160416', 'source': 'https://wiki.fintopia.tech/display/riskDev/2024H2', 'title': '2024H2', 'when': '2024-07-09T17:39:33.000+08:00'}, page_content='200ms KR3 模型推理框架国内外统一 【P2】 KR4 模型cache表优化，提高插入和查询效率，同时降低存储压力【P2】 建立HBase + Hive + PolarDB 3位一体的多级缓存 查询方面通过PolarDB + HBase提供高效查询，存储方面通过Hive提供长期存储功能，方面回溯各种问题 建立各级存储模块的过期策略，在PolarDB中存储原来的30%的数据，HBase存储未过期model的所有记录，hive存储最近3年的所有记录 O5 海外平台搭建 【P0】 以菲律宾、印尼为主 目标，训练全部迁移到ailab，推理迁移到新框架。'),
 Document(metadata={'id': '95160416', 'source': 'https://wiki.fintopia.tech/display/riskDev/2024H2', 'title': '2024H2', 'when': '2024-07-09T17:39:33.000+08:00'}, page_content='200ms KR3 模型推理框架国内外统一 【P2】 KR4 模型cache表优化，提高插入和查询效率，同时降低存储压力【P2】 建立HBase + Hive + PolarDB 3位一体的多级缓存 查询方面通过PolarDB + HBase提供高效查询，存储方面通过Hive提供长期存储功能，方面回溯各种问题 建立各级存储模块的过期策略，在PolarDB中存储原来的30%的数据，HBase存储未过期model的所有记录，hive存储最近3年的所有记录 O5 海外平台搭建 【P0】 以菲律宾、印尼为主 目标，训练全部迁移到ailab，推理迁移到新框架。'),
 Document(metadata={'id': '91856753', 'source': 'https://wiki.fintopia.tech/pages/vi

## 构建Rag chain

In [41]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
        [
            ("human", "你是一个解答用户问题的assistant，可以根据从语料库中召回的context文本回答问题。请尽量保证回答的内容都可以在context中找到根据，并务必保留 source 后面的url链接。\n以下是context文本：{context}，\n问题是：{question}"),
            ("assistant", "根据context文本，我认为答案是："),
        ]
    )

In [61]:
vectorstore.search("数据库", search_type="similarity", k=6)
# vectorstore.search?

[Document(metadata={'id': '91856753', 'source': 'https://wiki.fintopia.tech/pages/viewpage.action?pageId=91856753', 'title': '7. 在AiLab中连接hive', 'when': '2024-09-03T14:48:03.000+08:00'}, page_content='因为权限管理和LDAP一样，所以用户如果想要申请某个表的权限，需要去数仓申请 https://dw-lighthouse.fintopia.tech/tools/hive-permission 另外，如果不想把自己的密码保存在ipynb中，可以使用getpass函数，如下，这样就保存在passwd这个变量里了 ！！注意 ！！ hive.read_sql 这句话会把整个表下载到本地。强烈建议先在SQL中对数据进行过滤，采样或者group。 比如如果想知道某个表的行数，我们应该这样做： py 而不是这样：否则很可能把AiLab打挂 py 向hive中导入表 py ",'),
 Document(metadata={'id': '97182399', 'source': 'https://wiki.fintopia.tech/pages/viewpage.action?pageId=97182399', 'title': '模型迁移ModelServe', 'when': '2024-09-09T19:29:40.000+08:00'}, page_content='优缺点:直接读库开发简单,但是需要频繁扫model_result大表(百亿级),可能会有性能问题. 方案一:直接通过hivesql,通过hive直接对比增量数据 优缺点:通过hive的资源去做对比避免影响线上mysql性能,但是需要注意大批量的对比sql可能会超时. 5.一键切换模型服务到新 通过mirror服务核对后,确认模型迁移后无diff,需要提供一键转换功能,将模型的部署方式转换为ModelServe部署,主要更新MODEL_DATA中的deployType为3.然后清楚掉模型的缓存,让新的预测重新从数据库加载进行'),
 Document(metadata={'id': '97182399', 'sour

In [95]:
from langchain_core.retrievers import BaseRetriever
from langchain.vectorstores import VectorStore
from langchain.llms.base import LLM
from pydantic import Field

class VectorStoreRetrieverMoreHistory(BaseRetriever):
    vs: VectorStore
    llm: LLM
    topk: int = Field(gt=0, default=5, description="Number of documents to retrieve")

    def _get_relevant_documents(
        self, chat_str: str, *, run_manager):
        ai_ans = [c['content'] for c in self.llm.conversation_history if c['role'] == 'assistant']
        user_quest = self.llm.user_question_history
        qa_pairs = []
        for q, a in zip(user_quest, ai_ans):
            qa_pairs += [q, a]
            
        if chat_str.startswith(".. "):
        # append the last 2 conversation 
            query = "\n".join(qa_pairs[-2:] + [chat_str[3:]])
            self.llm.add_user_question(chat_str[3:])
        elif chat_str.startswith(".... "):
            # append all conversation history
            query = "\n".join(qa_pairs + [chat_str[5:]])
            self.llm.add_user_question(chat_str[5:])
        else:
            self.llm.clear_history()
            query = chat_str
            self.llm.add_user_question(query)
        return self.vs.similarity_search(query, k=self.topk)

retriever = VectorStoreRetrieverMoreHistory(vs=vectorstore, llm=llm, topk=8)


In [96]:
def format_docs(docs):
    return "\n\n".join(f"{d.page_content}\nsource: {d.metadata['source']}" for d in docs) 

llm.clear_history()
    
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [97]:
rag_chain.invoke("公共资源池和私有资源池的区别")

'公共资源池和私有资源池的区别主要在于计费方式、资源分配速度和使用优先级。\n\n1. 计费方式：\n  * 公共资源池是后付费的，根据小时数计费，不用不计费。\n  * 私有资源池是预付费的，即钱已经花出去了，不用就浪费了。source:https://wiki.fintopia.tech/pages/viewpage.action?pageId=80187151\n2. 资源分配速度：\n  * 公共资源池：创建和启动实例比较慢。\n  * 私有资源池：创建和启动实例快。source:https://wiki.fintopia.tech/pages/viewpage.action?pageId=80187151\n3. 使用优先级：\n  * 当用户提交任务时，系统会优先选择私有资源池的资源，如果私有资源池没有资源了，才会选择公有资源池的CPU机器。source:https://wiki.fintopia.tech/pages/viewpage.action?pageId=106844570'

In [107]:
rag_chain.invoke("如何进行分布式LightGBM模型训练")

'进行分布式LightGBM模型训练，可以基于Ray框架来实现。Ray是一款支持LightGBM等多种模型框架的分布式训练机器学习框架。它利用LightGBM自身的分布式训练功能，为LightGBM提供了一套多机的分布式计算框架。在分布式训练时，Ray会调用LightGBM官方提供的训练API进行机器组网，并开启分布式训练。此外，为了提高GPU利用率和训练效率，可以在分布式模式下采用特定的训练算法，如voting。但需要注意，分布式训练可能会导致一定的精度损失。更多细节和原理可以参考LightGBM的官方文档：https://lightgbm.readthedocs.io/en/latest/Features.html#optimization-in-distributed-learning 。\nsource: https://wiki.fintopia.tech/pages/viewpage.action?pageId=97186155、https://wiki.fintopia.tech/pages/viewpage.action?pageId=106844570'

In [108]:
rag_chain.invoke(".. 需要具备哪些前提条件")

'为了使用自研的分布式LightGBM打分模版，需要具备以下两个前提条件：1是数据集必须是分片好的，而且分片个数等于并行pod数的整数倍；2是数据集必须是parquet格式。详情请参考：https://wiki.fintopia.tech/pages/viewpage.action?pageId=105312091'

In [109]:
rag_chain.invoke(".. 参数建议如何设置")

'参数设置需要根据具体的模型训练任务和数据集特性来决定。在LightGBM的分布式训练中，可以考虑调整诸如"bagging_freq", "max_bin", "min_data_in_bin", "min_data_in_leaf", "min_sum_hessian_in_leaf", "min_child_weight", "boosting_type", "objective", "metric"等参数。具体的参数值需要通过实验和调优来确定，可以使用Optuna等调参工具来帮助选择较优的参数组合。并没有固定的参数设置建议，因为每个数据集和任务都是独特的。\nsource: https://wiki.fintopia.tech/pages/viewpage.action?pageId=85651968、https://wiki.fintopia.tech/pages/viewpage.action?pageId=85650497'

In [101]:
llm.conversation_history

[{'role': 'user',
  'content': 'Human: 你是一个解答用户问题的assistant，可以根据从语料库中召回的context文本回答问题。请尽量保证回答的内容都可以在context中找到根据，并务必保留 source 后面的url链接。以下是context文本：LightGBM分布式训练（CPU、GPU） 基于Ray框架 原理 Ray是一款非常通用的机器学习框架，同时支持LightGBM，XGboost，pytorch等多种模型框架的cpu，gpu及分布式训练。 它的LightGBM分布式训练方法本身也是利用了LightGBM自身的分布式训练功能，Ray只是为其提供了一套多机的分布式计算框架。关于LightGBM的分布式训练原理可以参考其官方文档： https://lightgbm.readthedocs.io/en/latest/Features.html#optimization-in-distributed-learning\nsource: https://wiki.fintopia.tech/pages/viewpage.action?pageId=97186155\n\n背景 为了加速大型数据集的LightGBM打分，我们自研了一套分布式LightGBM打分模版。它的原理非常简单，就是通过读取多个数据分片文件，然后把各个分片平均分配到多个pod，最后在每个pod上启动打分程序。所以，要想使用该模版， 需要具备2个前提条件 ，1是数据集必须是分片好的，而且分片个数等于并行pod数的整数倍，2是必须是parquet格式。为了方便大家使用，我们在automl_yqg这个包中，提供了2个函数进行分片，详见 下文 。 另外，分布式输出的打分文件是多个csv文件，可以借助automl_yqg.io中的read_dataframe这个函数进行读取。\nsource: https://wiki.fintopia.tech/pages/viewpage.action?pageId=105312091\n\n另外，在master节点，我们会通过ray client提交训练任务到这个集群上。 LightGBM分布式训练 则是Ray调用LightGBM官方提供的训练API进行机器组网，并开启分布式训练。 其实如果没有ray，只要我们有几台可以网

In [104]:
rag_chain.invoke("ailab实例一直黑屏")

'打开AILab一直停留在黑屏页面，大部分情况下，用户打开AILab应该可以直接进入Notebook页面。不过偶尔会卡在上述页面，此时点击一下右上角的关闭按钮即可。有的时候是因为登录过期了，这个字显示的非常不明显，需要自己看看。\nsource: https://wiki.fintopia.tech/display/riskDev/0.+FAQ'

In [110]:
ai_ans = [c['content'] for c in llm.conversation_history if c['role'] == 'assistant']
user_quest = llm.user_question_history
qa_pairs = []
for q, a in zip(user_quest, ai_ans):
    qa_pairs += [q, a]
qa_pairs

['如何进行分布式LightGBM模型训练',
 '进行分布式LightGBM模型训练，可以基于Ray框架来实现。Ray是一款支持LightGBM等多种模型框架的分布式训练机器学习框架。它利用LightGBM自身的分布式训练功能，为LightGBM提供了一套多机的分布式计算框架。在分布式训练时，Ray会调用LightGBM官方提供的训练API进行机器组网，并开启分布式训练。此外，为了提高GPU利用率和训练效率，可以在分布式模式下采用特定的训练算法，如voting。但需要注意，分布式训练可能会导致一定的精度损失。更多细节和原理可以参考LightGBM的官方文档：https://lightgbm.readthedocs.io/en/latest/Features.html#optimization-in-distributed-learning 。\nsource: https://wiki.fintopia.tech/pages/viewpage.action?pageId=97186155、https://wiki.fintopia.tech/pages/viewpage.action?pageId=106844570',
 '需要具备哪些前提条件',
 '为了使用自研的分布式LightGBM打分模版，需要具备以下两个前提条件：1是数据集必须是分片好的，而且分片个数等于并行pod数的整数倍；2是数据集必须是parquet格式。详情请参考：https://wiki.fintopia.tech/pages/viewpage.action?pageId=105312091',
 '参数建议如何设置',
 '参数设置需要根据具体的模型训练任务和数据集特性来决定。在LightGBM的分布式训练中，可以考虑调整诸如"bagging_freq", "max_bin", "min_data_in_bin", "min_data_in_leaf", "min_sum_hessian_in_leaf", "min_child_weight", "boosting_type", "objective", "metric"等参数。具体的参数值需要通过实验和调优来确定，可以使用Optuna等调参工具来帮助选择较优的参数组合。并没有固定的参数设置建议，因为每个数据集和任务都是独特的。\nsou